<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

- The winner is LightGMB on CPU

In [1]:
from ngboost import NGBoost, NGBRegressor
from ngboost.distns import Exponential, Normal
from lightgbm.sklearn import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X,y = make_regression(1000,100)
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [2]:
%%time
lgb = LGBMRegressor(n_estimators=100,
                    max_depth=3,
                    verbose=-1,
                    learning_rate=.1,
                    force_col_wise=True,
                    min_child_samples=2,
                    device="cpu")
lgb.fit(X_train,y_train)
preds=lgb.predict(X_test)
mean_squared_error(y_test,preds)

CPU times: user 915 ms, sys: 330 ms, total: 1.24 s
Wall time: 110 ms


4263.687522151796

In [26]:
%%time
lgb = LGBMRegressor(n_estimators=100,
                    max_depth=3,
                    verbose=-1,
                    learning_rate=.1,
                    force_col_wise=True,
                    device="cpu")
ngb = NGBRegressor(Dist=Normal, Base=lgb,n_estimators=500).fit(X_train,y_train)
preds=ngb.predict(X_test)
dist = ngb.pred_dist(X_test)
mean_squared_error(y_test,preds), dist.params["scale"].mean() # 0.005928139301892113

[iter 0] loss=6.7270 val_loss=0.0000 scale=2.0000 norm=321.9833
[iter 100] loss=5.5562 val_loss=0.0000 scale=2.0000 norm=53.4676
[iter 200] loss=4.5837 val_loss=0.0000 scale=2.0000 norm=12.7870
[iter 300] loss=2.7074 val_loss=0.0000 scale=4.0000 norm=5.4873
[iter 400] loss=0.9483 val_loss=0.0000 scale=4.0000 norm=2.4091
CPU times: user 7min 39s, sys: 12.5 s, total: 7min 52s
Wall time: 39.8 s


(1970.9071838101336, 0.19776681952827818)

In [4]:
%%time
lgb = LGBMRegressor(n_estimators=100,
                    max_depth=3,
                    verbose=-1,
                    learning_rate=.1,
                    device="gpu")
lgb.fit(X_train,y_train)
preds=lgb.predict(X_test)
mean_squared_error(y_test,preds)

CPU times: user 2.6 s, sys: 348 ms, total: 2.94 s
Wall time: 260 ms


3672.888290282761

In [5]:
%%time
lgb = LGBMRegressor(n_estimators=100,
                    max_depth=3,
                    verbose=-1,
                    learning_rate=.1,
                    device="gpu")
lgb.fit(X_train,y_train)
ngb = NGBRegressor(Dist=Normal, Base=lgb,n_estimators=300).fit(X_train,y_train)
preds=ngb.predict(X_test)
preds_param = ngb.pred_param(X_test)
mean_squared_error(y_test,preds)

[iter 0] loss=6.7270 val_loss=0.0000 scale=2.0000 norm=321.9833
[iter 100] loss=5.5562 val_loss=0.0000 scale=2.0000 norm=53.4676
[iter 200] loss=4.5837 val_loss=0.0000 scale=2.0000 norm=12.7870
CPU times: user 20min 52s, sys: 2min 10s, total: 23min 2s
Wall time: 1min 58s


1985.662547903987

In [6]:
%%time
xgb = XGBRegressor(n_estimators=100, 
                   max_depth=3, 
                   verbosity=0, 
                   learning_rate=.1,
                   tree_method="hist")
xgb.fit(X_train,y_train)
preds = xgb.predict(X_test)
mean_squared_error(y_test,preds)

CPU times: user 2.75 s, sys: 105 ms, total: 2.85 s
Wall time: 238 ms


4304.198169895188

In [7]:
%%time
xgb = XGBRegressor(n_estimators=100, 
                   max_depth=3, 
                   verbosity=0, 
                   learning_rate=.1,
                   tree_method="hist")
ngb = NGBRegressor(Dist=Normal, Base=xgb,n_estimators=300).fit(X_train,y_train)
preds=ngb.predict(X_test)
preds_param = ngb.pred_param(X_test)
mean_squared_error(y_test,preds)

/home/sergey/anaconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


[iter 0] loss=6.7270 val_loss=0.0000 scale=2.0000 norm=321.9833
[iter 100] loss=5.5507 val_loss=0.0000 scale=2.0000 norm=52.6475
[iter 200] loss=4.5722 val_loss=0.0000 scale=2.0000 norm=11.7392
CPU times: user 27min 55s, sys: 57.2 s, total: 28min 52s
Wall time: 2min 25s


2568.008642924577

In [8]:
%%time
xgb = XGBRegressor(n_estimators=100, 
                   max_depth=3, 
                   verbosity=0, 
                   learning_rate=.1,
                   tree_method="gpu_hist")
xgb.fit(X_train,y_train)
preds = xgb.predict(X_test)
mean_squared_error(y_test, preds)

CPU times: user 2.25 s, sys: 208 ms, total: 2.46 s
Wall time: 251 ms


4304.19823614987

In [9]:
%%time
xgb = XGBRegressor(n_estimators=100, 
                   max_depth=3, 
                   verbosity=0, 
                   learning_rate=.1,
                   tree_method="gpu_hist")
xgb.fit(X_train,y_train)
ngb = NGBRegressor(Dist=Normal, Base=xgb,n_estimators=300).fit(X_train,y_train)
preds=ngb.predict(X_test)
preds_param = ngb.pred_param(X_test)
mean_squared_error(y_test,preds)

/home/sergey/anaconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


[iter 0] loss=6.7270 val_loss=0.0000 scale=2.0000 norm=321.9833
[iter 100] loss=5.5507 val_loss=0.0000 scale=2.0000 norm=52.6475
[iter 200] loss=4.5722 val_loss=0.0000 scale=2.0000 norm=11.7326
CPU times: user 15min 17s, sys: 1min 9s, total: 16min 26s
Wall time: 1min 22s


2569.341305369871

In [30]:
%%time
cat = CatBoostRegressor(iterations=100, 
                        learning_rate=0.1, 
                        max_depth=3,
                        verbose=0,
                        task_type="CPU")
cat.fit(X_train, y_train)
preds = cat.predict(X_test)
mean_squared_error(y_test, preds)

CPU times: user 1.46 s, sys: 30.9 ms, total: 1.49 s
Wall time: 409 ms


2755.2483292568486

In [31]:
%%time
cat = CatBoostRegressor(iterations=100, 
                        learning_rate=0.1, 
                        max_depth=3,
                        verbose=0,
                        task_type="CPU")
ngb = NGBRegressor(Dist=Normal, Base=cat,n_estimators=300).fit(X_train,y_train)
preds=ngb.predict(X_test)
preds_param = ngb.pred_param(X_test)
mean_squared_error(y_test,preds)

[iter 0] loss=6.7270 val_loss=0.0000 scale=2.0000 norm=321.9833
[iter 100] loss=5.6152 val_loss=0.0000 scale=2.0000 norm=65.4091
[iter 200] loss=4.6990 val_loss=0.0000 scale=2.0000 norm=22.4405
CPU times: user 15min 18s, sys: 22.2 s, total: 15min 40s
Wall time: 3min 44s


649.1095010371579

In [32]:
%%time
cat = CatBoostRegressor(iterations=100, 
                        learning_rate=0.1, 
                        max_depth=3,
                        verbose=0,
                        task_type="GPU")
cat.fit(X_train, y_train)
preds = cat.predict(X_test)
mean_squared_error(y_test,preds)

CPU times: user 455 ms, sys: 129 ms, total: 584 ms
Wall time: 455 ms


2980.1152619176582

In [33]:
%%time
cat = CatBoostRegressor(iterations=100, 
                        learning_rate=0.1, 
                        max_depth=3,
                        verbose=0,
                        task_type="GPU")
cat.fit(X_train, y_train)
ngb = NGBRegressor(Dist=Normal, Base=cat,n_estimators=300).fit(X_train,y_train)
preds=ngb.predict(X_test)
preds_param = ngb.pred_param(X_test)
mean_squared_error(y_test,preds)

[iter 0] loss=6.7270 val_loss=0.0000 scale=2.0000 norm=321.9833
[iter 100] loss=5.6012 val_loss=0.0000 scale=2.0000 norm=62.9621
[iter 200] loss=4.6709 val_loss=0.0000 scale=2.0000 norm=20.4710
CPU times: user 5min 14s, sys: 1min 11s, total: 6min 25s
Wall time: 6min 9s


812.9709068285159